In [1]:
!pip install gradio
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, LSTM, Dense, Embedding, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset from Google Drive
file_path = '/content/drive/MyDrive/Dataset/train/train.csv'  # Update this with your file path
df = pd.read_csv(file_path)

# Preprocessing: Handle missing values
df = df.dropna(subset=['comment_text'])  # Drop rows with missing comments
df['comment_text'] = df['comment_text'].astype(str)  # Ensure all comments are strings

# Extract features (text) and labels
x = df['comment_text']
y = df.iloc[:, 2:].values  # Labels are from columns 2 onwards

# Ensure labels are binary (if not already)
y = np.where(y > 0, 1, 0)

# Parameters
MAX_FEATURES = 20000  # Max unique words in vocabulary
SEQ_LENGTH = 100  # Length of sequences after vectorization

# Text preprocessing function (optional improvements)
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.replace('\n', ' ')  # Remove newlines
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])  # Remove punctuation
    return text

# Apply preprocessing
x = x.apply(preprocess_text)

# Text vectorization
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=SEQ_LENGTH,
    output_mode='int'
)
vectorizer.adapt(x.values)
vectorized_text = vectorizer(x.values)

# Convert the vectorized text into a NumPy array for compatibility with train_test_split
vectorized_text = np.array(vectorized_text)

# Split dataset into train, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(vectorized_text, y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.33, random_state=42)

# Create TensorFlow datasets for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(16)

# Build the model with LSTM and Embedding
model = tf.keras.Sequential([
    Input(shape=(SEQ_LENGTH,)),
    Embedding(MAX_FEATURES + 1, 32),  # Embedding layer
    LSTM(64, activation='tanh', return_sequences=False),  # LSTM layer
    Dense(128, activation='relu'),  # Fully connected layer
    Dense(y.shape[1], activation='sigmoid')  # Output layer for multi-label classification
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(name="auc")]
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_dataset,
    epochs=5,  # Number of epochs
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

# Save the model
model_path = 'toxicity_model.keras'
model.save(model_path)

# Reload the model for prediction
model = tf.keras.models.load_model(model_path)

# Function to score a comment
def score_comment(comment):
    if not isinstance(comment, str) or not comment.strip():
        return "Invalid input. Please enter a valid comment."

    # Preprocess and vectorize the comment
    processed_comment = preprocess_text(comment)
    vectorized_comment = vectorizer([processed_comment])
    results = model.predict(vectorized_comment)

    # Format the output
    response = "Toxicity Classification:\n"
    labels = df.columns[2:].values  # Column names representing toxicity categories
    for idx, label in enumerate(labels):
        response += f"{label}: {'Toxic' if results[0][idx] > 0.5 else 'Non-Toxic'}\n"

    return response

# Optional: If you want to use Gradio for the user interface
import gradio as gr

interface = gr.Interface(
    fn=score_comment,
    inputs=gr.Textbox(lines=2, placeholder='Enter comment here...'),
    outputs='text',
    title="Toxic Comment Classifier",
    description=(
        "This model classifies comments into multiple toxicity categories. "
        "Enter a comment to see if it is toxic, obscene, a threat, etc."
    ),
    examples=[  # Example comments
        ["You're an idiot!"],
        ["Have a great day!"],
        ["I will kill you!"],
        ["I love you."]
    ]
)

# Launch the Gradio interface
interface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Mounted at /content/drive
Epoch 1/5
6982/6982 ━━━━━━━━━━━━━━━━━━━━ 408s 58ms/step - accuracy: 0.9887 - auc: 0.7998 - loss: 0.1361 - val_accuracy: 0.9940 - val_auc: 0.9758 - val_loss: 0.0573
Epoch 2/5
6982/6982 ━━━━━━━━━━━━━━━━━━━━ 441s 58ms/step - accuracy: 0.9932 - auc: 0.9762 - loss: 0.0530 - val_accuracy: 0.9940 - val_auc: 0.9791 - val_loss: 0.0528
Epoch